In [8]:
import sys
!{sys.executable} -m pip install numpy

In [11]:
import numpy
print(numpy.__path__)

_NamespacePath(['/usr/local/lib/python3.7/site-packages/numpy', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy'])


In [12]:
import matplotlib
print(matplotlib.__path__)

AttributeError: module 'numpy' has no attribute '__version__'

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from stocker import Stocker

AttributeError: module 'numpy' has no attribute '__version__'

In [ ]:
microsoft = Stocker('MSTF')

In [ ]:
stock_history = microsoft.stock
stock_history.head()

In [ ]:
stock_history['']

In [41]:
microsoft.plot_stock()

,Open,High,Low,Close
Date,,,,
2008-07-08,NaN,NaN,NaN,1415.09
2008-07-09,NaN,NaN,NaN,1471.62
2008-07-10,NaN,NaN,NaN,1486.05
2008-07-11,NaN,NaN,NaN,1446.56
2008-07-14,NaN,NaN,NaN,1460.94


In [ ]:
microsoft.plot_stock(start_date = "2000-01-03", end_date="2018-01-16", stats = ['Daily Change', 'Adj. Volume'], plot_type='pct')

In [ ]:
microsoft.buy_and_hold(start_date='1986-03-13', end_date='2020-01-19', nshares=100)

In [ ]:
microsoft.buy_and_hold(start_date='1999-01-05', end_date='2002-01-03', nshares=100)

In [ ]:
model, model_data = microsoft.create_prophet_model()

In [ ]:
model.plot_components(model_data)
plt.show()

In [ ]:
print(microsoft.weekly_seasonality)
microsoft.weekly_seasonality = True
print(microsoft.weekly_seasonality)

In [ ]:
model, model_data = microsoft.create_prophet_model(days=0)

In [ ]:
model.plot_components(model_data)
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
def get_preds_lin_reg(df, target_col, N, pred_min, offset):
    """
    Given a dataframe, get prediction at each timestep
    Inputs
        df         : dataframe with the values you want to predict     
        target_col : name of the column you want to predict
        N          : use previous N values to do prediction
        pred_min   : all predictions should be >= pred_min
        offset     : for df we only do predictions for df[offset:]
    Outputs
        pred_list  : the predictions for target_col
    """
    # Create linear regression object
    regr = LinearRegression(fit_intercept=True)
    pred_list = []
    for i in range(offset, len(df['Adj Close'])):
        X_train = np.array(range(len(df['Adj Close'][i-N:i]))) 
        y_train = np.array(df['Adj Close'][i-N:i]) 
        X_train = X_train.reshape(-1, 1)     
        y_train = y_train.reshape(-1, 1)
        regr.fit(X_train, y_train)            # Train the model
        pred = regr.predict(N)
    
        pred_list.append(pred[0][0])  
    
    # If the values are < pred_min, set it to be pred_min
    pred_list = np.array(pred_list)
    pred_list[pred_list < pred_min] = pred_min
        
    return pred_list
print(get_preds_lin_reg(df, df['Close'],df['Open'] ,df['Low'], 200))

In [ ]:
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
def get_mape(y_true, y_pred): 
    """
    Compute mean absolute percentage error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def train_pred_eval_model(X_train_scaled, \
                          y_train_scaled, \
                          X_test_scaled, \
                          y_test, \
                          col_mean, \
                          col_std, \
                          seed=100, \
                          n_estimators=100, \
                          max_depth=3, \
                          learning_rate=0.1, \
                          min_child_weight=1, \
                          subsample=1, \
                          colsample_bytree=1, \
                          colsample_bylevel=1, \
                          gamma=0):
    '''
    Train model, do prediction, scale back to original range and do 
    evaluation
    Use XGBoost here.
    Inputs
        X_train_scaled     : features for training. Scaled to have 
                             mean 0 and variance 1
        y_train_scaled     : target for training. Scaled to have 
                             mean 0 and variance 1
        X_test_scaled      : features for test. Each sample is 
                             scaled to mean 0 and variance 1
        y_test             : target for test. Actual values, not 
                             scaled
        col_mean           : means used to scale each sample of 
                             X_test_scaled. Same length as 
                             X_test_scaled and y_test
        col_std            : standard deviations used to scale each 
                             sample of X_test_scaled. Same length as 
                             X_test_scaled and y_test
        seed               : model seed
        n_estimators       : number of boosted trees to fit
        max_depth          : maximum tree depth for base learners
        learning_rate      : boosting learning rate (xgb’s “eta”)
        min_child_weight   : minimum sum of instance weight(hessian) 
                             needed in a child
        subsample          : subsample ratio of the training 
                             instance
        colsample_bytree   : subsample ratio of columns when 
                             constructing each tree
        colsample_bylevel  : subsample ratio of columns for each 
                             split, in each level
        gamma              : minimum loss reduction required to make 
                             a further partition on a leaf node of 
                             the tree
    Outputs
        rmse               : root mean square error of y_test and 
                             est
        mape               : mean absolute percentage error of 
                             y_test and est
        est                : predicted values. Same length as y_test
    '''
    model = XGBRegressor(seed=model_seed,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         learning_rate=learning_rate,
                         min_child_weight=min_child_weight,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree,
                         colsample_bylevel=colsample_bylevel,
                         gamma=gamma)
    
    # Train the model
    model.fit(X_train_scaled, y_train_scaled)
    
    # Get predicted labels and scale back to original range
    est_scaled = model.predict(X_test_scaled)
    est = est_scaled * col_std + col_mean
    # Calculate RMSE
    rmse = math.sqrt(mean_squared_error(y_test, est))
    mape = get_mape(y_test, est)
    
    return rmse, mape, est